# Advanced Machine Learning GR5242 - HW5

Author: Jie Li
Uni: jl5246

In [1]:
import pandas as pd
import numpy as np

## Problem 1: Gridworld value

#### quesition (a)

In [10]:
T = 200
r = 0
for i in range(T):
    if i % 5 == 1:
        r += 10 * 0.9**(i - 1)
round(r, 3)

24.419

#### quesition (b)

$$r = 10 + 0*4 + 100.9^{5} + 0*4 + 100.9^{10} + .... = 10*(1 + 0.9^{5} + 0.9^{10} + ...)= 10 * \sum_k^{200} 0.9^{5k}=10/(1-0.9^5) \approx 24.4 $$

#### quesition (c)

There are only some steps to reach A and return the largest rewards by 10

In [13]:
v_a = 24.4
for i in range(1, 7):
    print(round(0.9**i*v_a,1))

22.0
19.8
17.8
16.0
14.4
13.0


## Problem 2: Text Classification with an RNN

#### quesition (a)

In [15]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [17]:
# Download the dataset using TFDS
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [18]:
encoder = info.features['text'].encoder
print ('Vocabulary size: {}'.format(encoder.vocab_size))

sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print ('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('The original string: "{}"'.format(original_string))

assert original_string == sample_string

for index in encoded_string:
  print ('{} ----> {}'.format(index, encoder.decode([index])))


Vocabulary size: 8185
Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."
4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


In [20]:
# prepare training
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)

AttributeError: 'ShuffleDataset' object has no attribute 'output_shapes'

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=5,
                    validation_data=test_dataset, 
                    validation_steps=30)